In [ ]:
!pip install --force-reinstall dist/alquimia_fair_forge-0.0.1.tar.gz -q

In [ ]:
from fair_forge.metrics import Context
from pydantic import SecretStr
from helpers.dataset import CustomRetriever

In [8]:
from getpass import getpass
judge_api_key = SecretStr(getpass("Please enter your Judge API key: "))

In [9]:
metrics = Context.run(
    CustomRetriever,
    judge_api_key=judge_api_key
)

In [ ]:
def recreate_index(index_name: str, mapping: dict):
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
        print(f"Index '{index_name}' deleted.")
    es.indices.create(index=index_name, body=mapping)
    print(f"Index '{index_name}' created.")

In [ ]:
es = Elasticsearch(
    ELASTIC_URL,
    basic_auth=tuple(ELASTIC_AUTH),
)

In [ ]:
mapping_contextualizer = {
  "mappings": {
    "properties": {
      "session_id": {"type": "keyword"},
      "context": {"type": "text"},
      "context_insight": {"type": "text"},
      "context_awareness": {"type": "float"},
      "context_thinkings": {"type": "text"},
      "qa_id": {"type": "keyword"},
      "assistant_id": {"type": "keyword"},
    }
  }
}
recreate_index(context_index, mapping_contextualizer)

In [ ]:
docs = []
for metric in metrics:
    docs.append({
            "_index": context_index,
            "_source": metric.model_dump()
    })

helpers.bulk(es, docs)
print(f"Indexed {len(docs)} documents.")